# DataFrameIt - Exemplo 01: Uso Básico

Este notebook demonstra o uso básico do DataFrameIt para análise de sentimento.

**Conceitos demonstrados:**
- Criação de modelo Pydantic simples
- Template de prompt básico
- Processamento de dados sintéticos
- Uso padrão com LangChain/Gemini

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/bdcdo/dataframeit/blob/main/example/01_basic.ipynb)

## 1. Instalação

Execute a célula abaixo para instalar o DataFrameIt e suas dependências.

In [ ]:
# Instalar DataFrameIt com suporte ao Google Gemini
!pip install -q dataframeit[google]

## 2. Configuração da API Key

Configure sua chave de API do Google. No Google Colab, você pode usar o Secrets para armazenar sua chave de forma segura.

In [ ]:
import os

# Opção 1: Usando Google Colab Secrets (recomendado)
try:
    from google.colab import userdata
    os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')
    print("API Key carregada dos Secrets do Colab")
except:
    pass

# Opção 2: Inserir manualmente (descomente e preencha)
# os.environ['GOOGLE_API_KEY'] = 'sua-chave-aqui'

# Verificar se a chave foi configurada
if 'GOOGLE_API_KEY' not in os.environ:
    print("AVISO: Configure sua GOOGLE_API_KEY antes de continuar")
else:
    print("API Key configurada com sucesso!")

## 3. Imports

In [ ]:
from pydantic import BaseModel, Field
from typing import Literal
import pandas as pd
from dataframeit import dataframeit

## 4. Definir Modelo Pydantic

O modelo Pydantic define a estrutura dos dados que queremos extrair.

In [ ]:
class SentimentAnalysis(BaseModel):
    """Estrutura para análise de sentimento de textos."""

    sentimento: Literal['positivo', 'negativo', 'neutro'] = Field(
        ...,
        description="Classificação do sentimento geral do texto"
    )

    confianca: Literal['alta', 'media', 'baixa'] = Field(
        ...,
        description="Nível de confiança na classificação"
    )

    tema_principal: str = Field(
        ...,
        description="Tema ou assunto principal mencionado no texto"
    )

## 5. Definir Template do Prompt

O template define como o LLM deve processar cada texto.

In [ ]:
TEMPLATE = """
Você é um analista de sentimentos especializado.

Analise o texto abaixo e classifique o sentimento expresso.

Texto a analisar:
{texto}

Responda de forma estruturada seguindo o formato solicitado.
"""

## 6. Criar Dados Sintéticos

In [ ]:
dados = {
    'id': [1, 2, 3, 4, 5],
    'texto': [
        "Adorei o produto! A qualidade é excelente e chegou antes do prazo.",
        "Péssima experiência. O atendimento foi horrível e o produto veio com defeito.",
        "O produto é ok, nada de especial. Atende o básico.",
        "Estou muito feliz com a compra! Recomendo para todos os meus amigos.",
        "Não gostei. O preço é alto demais para a qualidade oferecida."
    ]
}

df = pd.DataFrame(dados)

print("DataFrame Original:")
df

## 7. Processar com DataFrameIt

O dataframeit vai:
1. Pegar cada texto da coluna 'texto'
2. Enviar para o LLM com o template
3. Extrair as informações no formato do modelo Pydantic
4. Adicionar as novas colunas ao DataFrame

In [ ]:
df_resultado = dataframeit(
    df,                      # DataFrame a processar
    SentimentAnalysis,       # Modelo Pydantic com estrutura esperada
    TEMPLATE,                # Template do prompt
    text_column='texto',     # Coluna que contém os textos (padrão: 'texto')
    resume=True              # Permite continuar de onde parou se interrompido
)

## 8. Visualizar Resultados

In [ ]:
# Mostrar resultados
colunas_exibir = ['id', 'sentimento', 'confianca', 'tema_principal']
df_resultado[colunas_exibir]

## 9. Estatísticas

In [ ]:
# Contar sentimentos
print("Distribuição de sentimentos:")
print(df_resultado['sentimento'].value_counts())

print("\nDistribuição de confiança:")
print(df_resultado['confianca'].value_counts())

## 10. Salvar Resultados (Opcional)

In [ ]:
# Descomente para salvar
# df_resultado.to_csv('resultado_sentimentos.csv', index=False)
# df_resultado.to_excel('resultado_sentimentos.xlsx', index=False)
# print("Resultados salvos!")

---

## Próximos Passos

- [02_error_handling.ipynb](02_error_handling.ipynb) - Aprenda a tratar erros
- [03_resume.ipynb](03_resume.ipynb) - Processamento incremental
- [05_advanced_legal.ipynb](05_advanced_legal.ipynb) - Exemplo avançado